In [3]:
import pandas as pd
from rbm_torch.utils import data_prep as dp
from rbm_torch.utils.utils import fasta_read
import rbm_torch.analysis.analysis_methods as am
from rbm_torch.utils.seq_utils import prune_similar_sequences

/home/jonah/anaconda3/envs/aptamer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
exo_df = am.fetch_data(["r2", "r3", "r4", "r5"], dir="./exo/", threads=6, molecule="dna")
exo_ct = dp.copynum_topology_faster(exo_df, ["r2", "r3", "r4", "r5"])
exo_ct.to_csv("./exo/exo_ct.csv", index=False)

In [4]:
exo_df = am.fetch_data(["r2", "r3", "r4", "r5"], dir="./exo/raw_rounds/", threads=6, molecule="dna")

Process Time 0.7867586612701416
Process Time 0.36142635345458984
Process Time 0.17887234687805176
Process Time 0.1158604621887207


In [5]:
rpm_dict = {}  # normalize counts to reads per million (added option in fasta_read to do this automatically)
for r in ["r2", "r3", "r4", "r5"]:
    round_data = exo_df[exo_df["round"] == r]
    rpm_dict[r] = round_data["copy_num"].sum()/1000000

In [7]:
rpm_dict

{'r2': 9.232675, 'r3': 10.478473, 'r4': 10.173198, 'r5': 8.49302}

In [ ]:
import pandas as pd
import numpy as np
exo_pd = pd.read_csv("./exo/exo_ct.csv")

# normalize counts
for r in ["r2", "r3", "r4", "r5"]:
    exo_pd[r] = exo_pd[r].div(rpm_dict[r])

# exo_pd["mean"] = exo_pd.apply(lambda row : np.nanmean(np.asarray([row[x] for x in ["r2", "r3", "r4", "r5"]])), axis=1)
# exo_pd["max"] = exo_pd.apply(lambda row : np.nanmax(np.asarray([row[x] for x in ["r2", "r3", "r4", "r5"]])), axis=1)


# def fold(dataframe, cols):
#     """ create fold column as col2/col1 for all columns"""
#     for cid, col in enumerate(cols):
#         for did, dol in enumerate(cols):
#             if cid >= did:
#                 continue
#             else:
#                 dataframe[f"{dol}/{col}_fold"] = dataframe.apply(lambda row: row[dol]/row[col] if row[col] != np.nan and row[dol] != np.nan else np.nan, axis=1)
#
#     return dataframe

# exo_pd = fold(exo_pd, ["r2", "r3", "r4", "r5"])

def enrichment_averge(df, round_names, min_diff=1, max_diff=None, diff_weights=None, round_weights=None):
    round_number = len(round_names)

    if max_diff is None:
        max_diff = round_number-1

    if diff_weights is None:
        diff_weights = [1. for x in range(min_diff, max_diff+1)]

    if round_weights is None:
        round_weights = [1. for x in range(len(round_names))]

    # first let's remove all the nan values in the dataframe, set nan values as the minimum normalized count for each round
    for r in round_names:
        df[r] = df[r].fillna(df[r].min())

    # Get fold value for round differences
    fold_keys = {diff: [] for diff in range(min_diff, max_diff+1)}
    for i in range(round_number):
        for j in range(round_number):
            if i >= j or j - i < min_diff or j - i > max_diff:
                continue
            fold_column_name = f"fold_{round_names[j]}v{round_names[i]}"
            fold_keys[j-i].append(fold_column_name)
            # fold_diffs.append(j-i)
            df[fold_column_name] = df[round_names[j]]/df[round_names[i]] * (round_weights[j] + round_weights[i])

    diff_keys = []
    for i in range(min_diff, max_diff+1):
        diff_avg_key = f"fold_diff{i}_avg"
        df[diff_avg_key] = df[fold_keys[i]].sum(axis=1).div(len(fold_keys[i])).mul(diff_weights[i-1])
        diff_keys.append(diff_avg_key)

    df["Final_Fold_Avg"] = df[diff_keys].sum(axis=1).div(len(diff_keys))

    return df

exo_pd = enrichment_averge(exo_pd, ["r2", "r3", "r4", "r5"], min_diff=1, max_diff=None, diff_weights=[0.95, 1.0, 1.0], round_weights=[0.11, 0.12,  0.13, 0.14])


In [ ]:
exo_pd["Fitness_Value"] = dp.scale_values_np(dp.log_scale(exo_pd["Final_Fold_Avg"].tolist(), base=1.0), min=0.01, max=1.0)

exo_pd.sort_values("Fitness_Value", ascending=False, inplace=True)
dp.dataframe_to_fasta(exo_pd, "./exo/fold_avg_all3.fasta", count_key="Fitness_Value")

In [ ]:
enriched_exo.sort_values("Final_Fold_Avg", ascending=False, inplace=True)
exo_enriched_pruned = prune_similar_sequences(enriched_exo, hamming_threshold=2, molecule="dna")

In [ ]:
exo_enriched_pruned = copy(exo_enriched_pruned)
exo_enriched_pruned["Fitness_Value"] = dp.scale_values_np(dp.log_scale(exo_enriched_pruned["Final_Fold_Avg"].tolist(), base=1.0), min=0.1, max=1.0)
dp.dataframe_to_fasta(exo_enriched_pruned, "./exo/enriched_trimmed.fasta", count_key="Fitness_Value")

In [ ]:
exo_pd[exo_pd["Final_Fold_Avg"] > 1].index.__len__()

In [ ]:
from copy import copy
enriched_exo = copy(exo_pd[exo_pd["Final_Fold_Avg"] > 1])

enriched_exo["Fitness_Value"] = dp.scale_values_np(dp.log_scale(enriched_exo["fold_r5vr2"].tolist(), base=1.0), min=0.1, max=1.0)

enriched_exo.sort_values("Fitness_Value", ascending=False, inplace=True)
dp.dataframe_to_fasta(enriched_exo, "./exo/enriched.fasta", count_key="Fitness_Value")

In [ ]:
from copy import copy
single_fold_exo = copy(exo_pd)

single_fold_exo["Fitness_Value"] = dp.scale_values_np(dp.log_scale(single_fold_exo["fold_r5vr2"].tolist(), base=1.0), min=0.01, max=1.0)

single_fold_exo.sort_values("Fitness_Value", ascending=False, inplace=True)
dp.dataframe_to_fasta(single_fold_exo, "./exo/fold_r5vr2.fasta", count_key="Fitness_Value")

In [ ]:
exo_pd.head()

In [24]:
enriched_exo.sort_values("Final_Fold_Avg", ascending=False, inplace=True)
exo_enriched_pruned = prune_similar_sequences(enriched_exo, hamming_threshold=2, molecule="dna")

Kept 9956 of 12339


In [26]:
exo_enriched_pruned = copy(exo_enriched_pruned)
exo_enriched_pruned["Fitness_Value"] = dp.scale_values_np(dp.log_scale(exo_enriched_pruned["Final_Fold_Avg"].tolist(), base=1.0), min=0.1, max=1.0)
dp.dataframe_to_fasta(exo_enriched_pruned, "./exo/enriched_trimmed.fasta", count_key="Fitness_Value")

In [21]:
exo_pd[exo_pd["Final_Fold_Avg"] > 1].index.__len__()

12339

In [22]:
from copy import copy
enriched_exo = copy(exo_pd[exo_pd["Final_Fold_Avg"] > 1])

enriched_exo["Fitness_Value"] = dp.scale_values_np(dp.log_scale(enriched_exo["fold_r5vr2"].tolist(), base=1.0), min=0.1, max=1.0)

enriched_exo.sort_values("Fitness_Value", ascending=False, inplace=True)
dp.dataframe_to_fasta(enriched_exo, "./exo/enriched.fasta", count_key="Fitness_Value")

In [19]:
from copy import copy
single_fold_exo = copy(exo_pd)

single_fold_exo["Fitness_Value"] = dp.scale_values_np(dp.log_scale(single_fold_exo["fold_r5vr2"].tolist(), base=1.0), min=0.01, max=1.0)

single_fold_exo.sort_values("Fitness_Value", ascending=False, inplace=True)
dp.dataframe_to_fasta(single_fold_exo, "./exo/fold_r5vr2.fasta", count_key="Fitness_Value")

In [17]:
exo_pd.head()

,sequence,r2,r3,r4,r5,fold_r3vr2,fold_r4vr2,fold_r5vr2,fold_r4vr3,fold_r5vr3,fold_r5vr4,fold_diff1_avg,fold_diff2_avg,fold_diff3_avg,Final_Fold_Avg,Fitness_Value
222388,GGTCCTCAACCTGCGACGGAGTCCCAACATGTCCA---,0.108311,0.095434,0.196595,315185.882054,0.202655,0.435623,727502.203398,0.515004,858693.356321,4.328705e+05,137075.895715,429346.895972,727502.203398,431308.331695,1.000000
235661,TGTTACCTACGCATACCATTACGGGCCTTTGACTGTGA,0.108311,0.095434,0.196595,69177.159597,0.202655,0.435623,159672.557995,0.515004,188466.459754,9.500665e+04,30085.665515,94233.447689,159672.557995,94663.890400,0.883918
126562,GGAGCTATGCCGTGATGGGCTCTACTGTTGTTTTTA--,1.083110,0.477169,176580.854909,0.117744,0.101328,39127.527470,0.027177,92514.886024,0.064156,1.800354e-07,29296.412661,19563.795813,0.027177,16286.745217,0.749201
144680,ATCCAGCGTAGCACCCGACATCGCAACCCGTAGAAA--,1.083110,0.477169,109302.011029,14785.317826,0.101328,24219.598672,3412.700856,57265.908571,8056.232789,3.652299e-02,18134.248033,16137.915731,3412.700856,12561.621540,0.729323
21663,TTCCGACACCCTCAACGCAGGGTTGAACTTACCCCTGA,1.083110,0.477169,93698.952876,0.117744,0.101328,20762.207514,0.027177,49091.097392,0.064156,3.392867e-07,15545.546261,10381.135835,0.027177,8642.236425,0.700699


In [7]:
exo_pd.index.__len__()

242496

In [6]:
exo_pd.head()

,sequence,r2,r3,r4,r5,max,fold_r3vr2,fold_r4vr2,fold_r5vr2,fold_r4vr3,fold_r5vr3,fold_r5vr4,fold_diff1_avg,fold_diff2_avg,fold_diff3_avg,Final_Fold_Avg
0,TGCGGGGCAATTTGAACACACCCGCAATCCCAGTTTGA,3.465951,2.958446,0.098298,0.117744,3.465951,1.792506,0.060976,0.074737,0.074759,0.091538,2.814902,1.560722,0.091508,0.097159,0.583130
1,ATGGATCACAAGGTGTTTCTGTTTTTTTTGGGGTAA--,0.108311,0.095434,0.098298,0.235487,0.235487,1.850329,1.951230,4.783195,2.317517,5.675364,5.629803,3.265883,4.575956,6.218153,4.686664
2,TTTGAACGTCCGCAGCTGCAATCGGGCGCTTAGCCA--,2.057908,1.240639,0.098298,0.117744,2.057908,1.266014,0.102696,0.125874,0.178271,0.218283,2.814902,1.419729,0.192588,0.163636,0.591984
3,TGACGTAGTGACTGGATCTACACATTTTTCTTACT---,1.191421,0.477169,0.098298,0.117744,1.191421,0.841058,0.177385,0.217418,0.463503,0.567536,2.814902,1.373154,0.446953,0.282643,0.700917
4,ATTAAGTTGGTAGCCGCCACCATGTTTGTCAGATC---,0.974799,0.381735,4.718280,0.117744,4.718280,0.822368,10.406561,0.265733,27.810210,0.709420,0.058644,9.563741,6.669589,0.345453,5.526261


In [22]:
from copy import copy
enriched = copy(exo_pd[(exo_pd["r5/r4_fold"] > 2) | (exo_pd["r5/r3_fold"] > 3)])
enriched["fold"] = enriched.apply(lambda row: np.nanmax(np.asarray([row["r5/r4_fold"], row["r5/r3_fold"]])), axis=1)
print(enriched.index.__len__())

23382


In [23]:
enriched.sort_values("fold", ascending=False, inplace=True)
enriched_trimmed = prune_similar_sequences(enriched, hamming_threshold=4, molecule="dna")

Kept 20919 of 23382


In [24]:
dp.dataframe_to_fasta(enriched_trimmed, "./exo/enriched.fasta", count_key="fold")

In [25]:
seqs, folds, chars, q = fasta_read("./exo/enriched.fasta", "dna", threads=6)

Process Time 0.024022579193115234


In [36]:
std_folds = dp.standardize_affinities(folds, out_plots="./exo/enriched", scale="log", dividers=[10], target_scaling=[2.49], divider_type="percentile")

In [38]:
dp.make_weight_file("./exo/en_fold_st", std_folds)